In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../Data/athlete_events.csv')
region_df = pd.read_csv('../Data/noc_regions.csv')
tokyo_df = pd.read_csv('../Data/tokyo_dataset.csv', encoding='latin1')

In [3]:
df.head(3)

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN


In [4]:
tokyo_df.head(3)

,Unnamed: 0,Code,Name,Gender,Age,NOC,Country,Discipline,Sport,Event,Rank,Medal
0,1,1346266,AALERUD Katrine,Female,26,NOR,Norway,CRD,Cycling Road,Women's Road Race,37.0,NaN
1,2,1346266,AALERUD Katrine,Female,26,NOR,Norway,CRD,Cycling Road,Women's Individual Time Trial,20.0,NaN
2,3,1355250,ABAD Nestor,Male,28,ESP,Spain,GAR,Artistic Gymnastics,Men's All-Around,NaN,NaN


In [5]:
df = df[df['Season'] == 'Summer']
df = df.merge(region_df, on='NOC', how='left')
df.drop_duplicates(inplace=True)

df = df.rename(columns={'Sex':'Gender','region':'Country'})
df['Gender'] = df['Gender'].map( {'M':'Male','F':'Female',} )

df = df.drop(['ID','Gender','Age','Height','Weight','NOC','Season','notes'],axis=1)
df = df[['Name','Year','Country','Team','Games','City','Sport','Event','Medal']]
df.head(3)

,Name,Year,Country,Team,Games,City,Sport,Event,Medal
0,A Dijiang,1992,China,China,1992 Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,A Lamusi,2012,China,China,2012 Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,Gunnar Nielsen Aaby,1920,Denmark,Denmark,1920 Summer,Antwerpen,Football,Football Men's Football,NaN


In [6]:
tokyo_df = tokyo_df.drop(['Unnamed: 0','Gender','Age','NOC','Code','Discipline','Rank'],axis=1)
tokyo_df['Year'] = 2020
tokyo_df = tokyo_df[['Name','Year','Country','Sport','Event','Medal']]
tokyo_df.head(3)

,Name,Year,Country,Sport,Event,Medal
0,AALERUD Katrine,2020,Norway,Cycling Road,Women's Road Race,NaN
1,AALERUD Katrine,2020,Norway,Cycling Road,Women's Individual Time Trial,NaN
2,ABAD Nestor,2020,Spain,Artistic Gymnastics,Men's All-Around,NaN


In [7]:
result_df = pd.concat([df, tokyo_df], axis=0)
result_df.reset_index(drop=True, inplace=True)

In [8]:
x = pd.get_dummies(result_df['Medal']).astype(int)
result_df = pd.concat([result_df,x], axis=1)

result_df['Total'] = result_df['Gold'] + result_df['Silver'] + result_df['Bronze']
result_df = result_df[['Name','Year','Country','Team','Games','City','Sport','Event','Medal','Gold','Silver','Bronze','Total']]

In [9]:
result_df.head(3)

,Name,Year,Country,Team,Games,City,Sport,Event,Medal,Gold,Silver,Bronze,Total
0,A Dijiang,1992,China,China,1992 Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,0,0,0,0
1,A Lamusi,2012,China,China,2012 Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,0,0,0,0
2,Gunnar Nielsen Aaby,1920,Denmark,Denmark,1920 Summer,Antwerpen,Football,Football Men's Football,NaN,0,0,0,0


In [10]:
# result_df.to_csv('medal_count.csv', index=False)